In [1]:
#NN Surrogate model class
import injector_surrogate_quads
from injector_surrogate_quads import *
import physics_gp

sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

#Pytorch 
import numpy as np
import torch
import transformer
import gpytorch
import botorch 

# BO for Minimizing Emittance with 3 Variables (SQ, CQ, SOL)

In [2]:
#load injector model
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

2022-06-10 14:24:50.314180: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Objective Function

In [3]:
#convert to machine units
ref_point = Model.sim_to_machine(np.asarray(ref_point))

#input params: solenoid and quads to vary 
opt_var_names = ['SOL1:solenoid_field_scale','SQ01:b1_gradient','CQ01:b1_gradient']

#output params: emittance in transverse plane (x & y)
opt_out_names = ['norm_emit_x','norm_emit_y']

def evaluate(varx, vary, varz): 

    #make input array of length model_in_list (inputs model takes)
    x_in = np.empty((1,len(Model.model_in_list)))

    #fill in reference point around which to optimize
    x_in[:,:] = np.asarray(ref_point[0])

    #set solenoid, SQ, CQ to values from optimization step
    x_in[:, Model.loc_in[opt_var_names[0]]] = varx
    x_in[:, Model.loc_in[opt_var_names[1]]] = vary
    x_in[:, Model.loc_in[opt_var_names[2]]] = varz

    #output predictions
    y_out = Model.pred_machine_units(x_in) 

    return -1*objective(y_out)[0]


def objective(y_out):
    
    #output is geometric emittance in transverse plane
    out1 = y_out[:,Model.loc_out['norm_emit_x']] #grab norm_emit_x out of the model
    out2 = y_out[:,Model.loc_out['norm_emit_y']] #grab norm_emit_y out of the model
       
    return np.sqrt(out1*out2)/1e-6 # in um units

## Gaussian Regression & Acquisition Function

In [4]:
def get_BO_point(x, f, bounds, beta=2.5, precision = None, phys=True):
    """

    function that trains a GP model of data and returns the next observation point using UCB
    D is input space dimensionality
    N is number of samples

    :param x: input points data, torch.tensor, shape (N,D)
    :param f: output point data, torch.tensor, shape (N,1)
    :param bounds: input space bounds, torch.tensor, shape (2,D)
    :param precision: precision matrix used for RBF kernel (must be PSD), torch.tensor, (D,D)
    :param beta: UCB optimization parameter, float
    :return x_candidate, model: next observation point and gp model w/observations
    """
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    
    if phys == True:
        gp = physics_gp.PhysicsExactGPModel(x, f.flatten(), likelihood, precision)
    else:
        gp = botorch.models.SingleTaskGP(x, f, likelihood)
        
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(gp.likelihood, gp)
    
    # fit GP hyperparameters
    #print('training hyperparameters')
    botorch.fit.fit_gpytorch_model(mll)

    # do UCB acquisition
    #print('optimizing acquisition function')
    UCB = botorch.acquisition.UpperConfidenceBound(gp, beta=beta)
    candidate, acq_value = botorch.optim.optimize_acqf(UCB,
                                                       bounds=bounds,
                                                       q=1,
                                                       num_restarts=20,#5
                                                       raw_samples=20)
    return candidate, gp

## Set up initial training samples

In [5]:
#specify bounds
bounds = {'varx': (0.44, 0.55),
          'vary': (-0.02, 0.02),
          'varz': (-0.02, 0.02)
         }

tbounds = torch.tensor([[bounds['varx'][0], bounds['varx'][1]],[bounds['vary'][0], 
                         bounds['vary'][1]], [bounds['varz'][0], bounds['varz'][1]]])

#print(bounds['varx'][0])
#print(tbounds)

diff = {'varx': bounds['varx'][1] - bounds['varx'][0],
        'vary': bounds['vary'][1] - bounds['vary'][0],
        'varz': bounds['varz'][1] - bounds['varz'][0]
       }

#create initial samples within specified bounds
n_samples = 5
n_var = 3
train_x = torch.zeros((n_samples, n_var)) 
for i in range(n_var):
    train_x[:,i] = torch.as_tensor(np.random.uniform(tbounds[i,0],tbounds[i,1],(n_samples,)))
#train_x = torch.as_tensor(train_x).type(torch.FloatTensor)

"""
train_x = torch.rand(n_samples,3) 
train_x[:,0] = train_x[:,0] * diff['varx'] + bounds['varx'][0]
train_x[:,1] = train_x[:,1] * diff['vary'] + bounds['vary'][0]
train_x[:,2] = train_x[:,2] * diff['varz'] + bounds['varz'][0]
""" 
print(train_x)

train_y = torch.tensor([evaluate(vars[0], vars[1], vars[2]) for vars in train_x]).reshape(-1,1)
print(train_y)


#transformer 
transformer_x = transformer.Transformer(tbounds.transpose(0,1), transform_type = 'normalize')
print(transformer_x.mins)
print(transformer_x.maxs)
normed_train_x = transformer_x.forward(train_x)

transformer_y = transformer.Transformer(train_y, transform_type = 'standardize') 
normed_train_y = transformer_y.forward(train_y)

""" 
#create initial model
gp = SingleTaskGP(normed_train_x, normed_train_y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll);
"""

tensor([[ 0.4485,  0.0084, -0.0148],
        [ 0.4854,  0.0186,  0.0134],
        [ 0.5018,  0.0057, -0.0022],
        [ 0.4840, -0.0017,  0.0139],
        [ 0.5075,  0.0118,  0.0165]])


2022-06-10 14:24:51.090541: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-10 14:24:51.115156: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1996200000 Hz


tensor([[-1.5719],
        [-1.4858],
        [-2.1122],
        [-0.9520],
        [-2.9320]])
[ 0.44 -0.02 -0.02]
[0.55 0.02 0.02]


' \n#create initial model\ngp = SingleTaskGP(normed_train_x, normed_train_y)\nmll = ExactMarginalLogLikelihood(gp.likelihood, gp)\nfit_gpytorch_model(mll);\n'

## Bayesian Optimization

In [6]:
n_steps = 30
best_y = torch.max(train_y)
for i in range(n_steps):
    if i > 10:
        phys_l = True
        
    bounds = torch.cat((torch.zeros(1,n_var), torch.ones(1,n_var)), 0)
    #x_new, model = get_BO_point(train_x, normed_train_y, bounds=tbounds.transpose(0,1), phys = False)
    x_new, model = get_BO_point(normed_train_x, normed_train_y, bounds=bounds, phys = False)
    
    #train_x = torch.cat((train_x, x_new))
    train_x = torch.cat((train_x, transformer_x.backward(x_new)))
    normed_train_x = transformer_x.forward(train_x)
    
    new_y = torch.tensor(evaluate(train_x[-1,0], train_x[-1,1], train_x[-1,2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    normed_train_y = transformer_y.forward(train_y)
    
    train_x = train_x.type(torch.FloatTensor)
    train_y = train_y.type(torch.FloatTensor)
    
    if (new_y > best_y):
        best_y = new_y
        color = '\033[95m', '\033[0m'
    else: 
        color = '\u001b[30m', '\033[0m'
    
    print("iteration        target         varx         vary         varz")
    print(f'{color[0]}{i+1}              {train_y[-1,0]:.5f}      {train_x[-1,0]:.5f}      {train_x[-1,1]:.5f}      {train_x[-1,2]:.5f}{color[1]}')
    #print(f'max = {torch.max(train_y):.5f}')

iteration        target         varx         vary         varz
1              -1.11107      0.46631      0.00163      0.01349
iteration        target         varx         vary         varz
2              -1.31722      0.45806      -0.00710      0.01487
iteration        target         varx         vary         varz
3              -0.63842      0.47683      -0.01318      0.00531
iteration        target         varx         vary         varz
4              -0.60652      0.47524      -0.00386      -0.00453
iteration        target         varx         vary         varz
5              -1.09733      0.47709      -0.01943      -0.01331
iteration        target         varx         vary         varz
6              -0.64159      0.48163      -0.00628      -0.00056
iteration        target         varx         vary         varz
7              -0.57360      0.47617      -0.00331      0.00384
iteration        target         varx         vary         varz
8              -0.57811      0.47675      0.00

In [7]:
"""
#define acquisition function
from botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement
from botorch.optim import optimize_acqf

#optimize
n_steps = 45
for i in range(n_steps):
    #best_normed_y = torch.max(normed_train_y)
    UCB = UpperConfidenceBound(gp, beta=2.5)
    #EI = ExpectedImprovement(gp, best_normed_y)

    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)
    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)

    train_x = torch.cat((train_x, transformer_x.backward(candidate)))
    normed_train_x = transformer_x.forward(train_x)

    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)
    train_y = torch.cat((train_y, new_y))
    
    print("iteration        target         varx         vary         varz")
    print(f'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-1][2]:.5f}')
    print(torch.max(train_y))
    
    transformer_y = transformer.Transformer(train_y, 'standardize')
    normed_train_y = transformer_y.forward(train_y)

    gp = SingleTaskGP(normed_train_x, normed_train_y)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_model(mll);
"""

'\n#define acquisition function\nfrom botorch.acquisition.analytic import UpperConfidenceBound, ExpectedImprovement\nfrom botorch.optim import optimize_acqf\n\n#optimize\nn_steps = 45\nfor i in range(n_steps):\n    #best_normed_y = torch.max(normed_train_y)\n    UCB = UpperConfidenceBound(gp, beta=2.5)\n    #EI = ExpectedImprovement(gp, best_normed_y)\n\n    bounds = torch.cat((torch.zeros(1,3), torch.ones(1,3)), 0)\n    candidate, acq_value = optimize_acqf(UCB, bounds = bounds, num_restarts = 20, q = 1, raw_samples = 20)\n\n    train_x = torch.cat((train_x, transformer_x.backward(candidate)))\n    normed_train_x = transformer_x.forward(train_x)\n\n    new_y = torch.tensor(evaluate(train_x[-1][0], train_x[-1][1], train_x[-1][2])).reshape(1,1)\n    train_y = torch.cat((train_y, new_y))\n    \n    print("iteration        target         varx         vary         varz")\n    print(f\'{i+1}              {train_y[-1][0]:.5f}      {train_x[-1][0]:.5f}      {train_x[-1][1]:.5f}      {train_x[-